In [1]:
import pickle5 as pickle
import pandas as pd
import yfinance as yf

# S&P500에 속한 기업들 티커를 받기 위한 모듈 임포트
import bs4 as bs
import requests
import yfinance as yf
import datetime

In [2]:
# 가격 데이터 로드
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/spx_price.pkl', 'rb') as f:
    spx_price = pickle.load(f)
    spx_price.index = pd.to_datetime(spx_price.index)
    
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/commodity_price.pkl', 'rb') as f:
    commodity_price = pickle.load(f)
    commodity_price.index = pd.to_datetime(commodity_price.index)
    
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/usdkrw_price.pkl', 'rb') as f:
    usdkrw_price = pickle.load(f)
    usdkrw_price.index = pd.to_datetime(usdkrw_price.index)
    
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/oecd_cli.pkl', 'rb') as f:
    oecd_cli_df = pickle.load(f)
    oecd_cli_df.index = pd.to_datetime(oecd_cli_df.index)
    # 선견편향 제거
    oecd_cli_df = oecd_cli_df.shift(1)
    # 시점별 국가 수 계산
    noc = len(oecd_cli_df.columns) - oecd_cli_df.diff().isna().sum(axis=1)
    # 확산 지수
    diffusion_index = (oecd_cli_df.diff() > 0).sum(axis=1) / noc

bond_tickers = ['TLT', 'IEF']
bond_price = yf.Tickers(bond_tickers)
bond_price = bond_price.history(start='2010-01-01', actions=False)
bond_price.drop(['Open', 'High', 'Low', 'Volume'], inplace=True, axis=1)
bond_price = bond_price.droplevel(0, axis=1)#.resample('W').last()
bond_price.index = pd.to_datetime(bond_price.index.date)

[*********************100%***********************]  2 of 2 completed


In [2]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers_list = [s.replace('\n', '') for s in tickers]

start = datetime.datetime(2000,1,1)
end = datetime.datetime(2022,12,31)
df = yf.download(tickers_list, start=start, end=end)
sp_price = df['Adj Close']

sp_price.index = pd.to_datetime(pd.to_datetime(sp_price.index).date)
sp_price.index.name = 'date_time'

sp_price

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: AttributeError("'NoneType' object has no attribute 'tz_convert'")


,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date_time,,,,,,,,,,,,,,,,,,,,,
2000-01-03,44.001263,NaN,NaN,0.850643,NaN,2.988898,9.221025,1.277778,NaN,16.274672,...,NaN,7.294353,19.130636,6.772068,NaN,4.792224,NaN,25.027779,37.955753,NaN
2000-01-04,40.640064,NaN,NaN,0.778926,NaN,2.784836,8.957568,1.270833,NaN,14.909400,...,NaN,7.462312,18.764208,6.772068,NaN,4.695898,NaN,24.666668,36.117786,NaN
2000-01-05,38.119156,NaN,NaN,0.790324,NaN,3.000901,8.941102,1.388889,NaN,15.204175,...,NaN,7.750247,19.787149,6.889136,NaN,4.719979,NaN,25.138889,36.075039,NaN
2000-01-06,36.667706,NaN,NaN,0.721930,NaN,3.228970,9.253957,1.375000,NaN,15.328288,...,NaN,7.678261,20.810097,6.898145,NaN,4.679843,NaN,23.777779,36.587967,NaN
2000-01-07,39.723354,NaN,NaN,0.756128,NaN,3.649096,9.352757,1.451389,NaN,16.072985,...,NaN,7.678261,20.749020,6.880135,NaN,4.575492,NaN,23.513889,36.673454,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,149.007767,12.71,143.279999,131.658981,161.564163,169.487625,107.692398,63.380001,265.006012,338.450012,...,80.720001,70.446365,108.680000,31.701571,109.730003,128.899994,126.690002,248.220001,48.450001,145.418304
2022-12-27,149.327301,12.53,145.020004,129.831772,161.455200,168.480743,108.080643,63.619999,264.229187,335.089996,...,84.330002,71.082001,110.190002,31.940601,110.720001,129.899994,127.279999,251.000000,48.840000,144.959396
2022-12-28,147.869461,12.32,145.300003,125.847855,160.702347,166.845779,107.343979,62.599998,262.048126,328.329987,...,80.089996,70.570000,108.379997,30.855000,108.940002,129.309998,125.989998,246.839996,47.970001,143.492844


In [4]:
sp_price.to_csv('equity_universe.csv')

In [ ]:
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/oecd_cli.pkl', 'rb') as f:
    oecd_cli_df = pickle.load(f)
    oecd_cli_df.index = pd.to_datetime(oecd_cli_df.index)
    # 선견편향 제거
    oecd_cli_df = oecd_cli_df.shift(1)
    # 시점별 국가 수 계산
    noc = len(oecd_cli_df.columns) - oecd_cli_df.diff().isna().sum(axis=1)
    # 확산 지수
    diffusion_index = (oecd_cli_df.diff() > 0).sum(axis=1) / noc

In [ ]:
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/factor.pkl', 'rb') as f:
    df = pickle.load(f)
    regime = df.iloc[:, :5]
    regime.index = pd.to_datetime(regime.index)

In [ ]:
diffusion_index = pd.DataFrame({'Diffusion': diffusion_index})
diffusion_index

In [ ]:
econ_regime = pd.concat([diffusion_index, regime], axis=1)
#econ_regime = econ_regime.rename(columns={0: 'Diffusion'})
econ_regime.index = pd.to_datetime(econ_regime.index)
econ_regime.index = pd.to_datetime(econ_regime.index.date)
econ_regime.dropna(axis=0, inplace=True)

econ_regime.to_csv('econ_regime.csv')

In [ ]:
col_info_dict = {'equity': sp_price,
                'sp500 index': spx_index,
                'commodity': commodity_price,
                'fx': usdkrw_price,
                'bond etf': bond_price,
                'regime': econ_regime,
                }

In [ ]:
df_list = []

for key, df in col_info_dict.items():
    df_list.append(make_col_infos(df, name=key))

In [1]:
import pandas as pd
import yfinance as yf
from econ_regime.econ_regime import *

In [2]:
eq = pd.read_csv('equity_universe.csv', index_col=0).loc['2011':]
eq.index = pd.to_datetime(eq.index)
alter = asset_indicators()
alter.index = pd.to_datetime(alter.index)
rg = business_cycle()#.shift(1)
rg.index = pd.to_datetime(rg.index)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
alter.columns

Index(['SPY', 'TLT', 'GSG', 'VNQ', 'UUP'], dtype='object')

In [24]:
ben_equ_reg_df = pd.concat([eq, alter], axis=1)
ben_equ_reg_df.loc['2011': '2022'].dropna(axis=1).to_csv('alter_with_equity.csv')